# Web Scrapping 

## 1. Beatifulsoap

El **_web scraping_** es una técnica que permite **extraer información de sitios web** de forma automatizada.
En lugar de copiar datos manualmente desde una página, un programa “visita” esa página, descarga su código HTML, y luego extrae los fragmentos de información que interesan (por ejemplo, títulos, precios, autores, etc.).
Se basa en el análisis y extracción de datos de HTML/XML

Cuando ingresas una URL en el navegador:

* El navegador envía una solicitud HTTP (request) al servidor del sitio.

* El servidor responde con el HTML de la página.

* El navegador interpreta ese HTML y lo muestra visualmente.

El **scraper hace lo mismo**, pero en lugar de mostrar la página, **analiza el HTML y toma los datos** que necesita.

Es rápida, fácil de usar pero solo válida para pequeños proyectos

En el siguiente ejemplo vemos como Python hace un request a una página


In [1]:
import requests

# URL del sitio que queremos scrapear
url = "https://es.wikipedia.org/wiki/Python"

# Encabezados HTTP personalizados
# Es importante incluir un User-Agent identificable
headers = {
    "User-Agent": "MiScraperEducativo/1.0 (+https://tusitio.com/contacto) - uso académico"
}

# Realizamos la petición GET con los encabezados
response = requests.get(url, headers=headers)

# Comprobamos el código de estado HTTP
if response.status_code == 200:
    print("✅ Conexión exitosa. Página descargada correctamente.\n")
    # Mostramos los primeros 500 caracteres del HTML
    print(response.text[:500])
else:
    print(f"⚠️ Error {response.status_code}: No se pudo acceder a la página.")
    print(response.text[:200])


✅ Conexión exitosa. Página descargada correctamente.

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vect


A partir de aquí ya podríamos hacer scrapy. 
Pero antes hay que analizar **los aspectos legales y éticos** del webscraping y conocer las buenas prácticas:


* Revisar el archivo robots.txt del sitio, generalmente está en **_https://sitio.com/robots.txt_** para ver qué partes permiten ser scrapeadas.

* No hacer demasiadas peticiones seguidas (respetar el servidor). Muchas veces hasta te bloquea.

* No recolectar información personal o protegida.

* Usar siempre un User-Agent identificando al scraper.

El objetivo es aprender y automatizar tareas sin dañar sitios ni infringir derechos.

### 1. BeautifulSoup

**_BeautifulSoup_** es una librería de Python diseñada para analizar **_(parsear)_** documentos HTML o XML y extraer información específica de ellos.

Podrías pensar en ella como una herramienta que convierte el HTML en una estructura de árbol, donde puedes moverte fácilmente por las etiquetas y obtener los datos que te interesan.

Puede que necesitemos instalar **_pip install beautifulsoup4_**   y también requests **_pip install requests_**.

Empecemos por un ejemplo. 

In [2]:
## Paso 1.
## Primero vamos a averiguar si la página responde correctamente, suele ser con el valor 200

#importanciones y url diana
import requests
from bs4 import BeautifulSoup

url = "https://books.toscrape.com/"

#creamos un encabezado HTTP que identifica tu programa ante el servidor, indicando quién hace la petición (el “User-Agent”)
# para evitar bloqueos y cumplir con las buenas prácticas de scraping.

headers = {"User-Agent": "MiScraperEducativo/1.0 (+https://tusitio.com/contacto)"}

response = requests.get(url, headers=headers)
print("Código de estado:", response.status_code)

if response.status_code == 200:
    print("PERFECTO.... Ahora vamos a crear el objeto BeautifulSoup")

Código de estado: 200
PERFECTO.... Ahora vamos a crear el objeto BeautifulSoup


In [3]:
## PASO 2. Parsear
## response.txt es una cadena de texto PLANO que tiene todo el código HTML que descargamos con requests
## <!DOCTYPE html><html><head>...<body>...<div>...</div></body></html>
## Beatifulsoup html.parser, Beatifulsoup toma ese texto y lo parsea creando el objeto DOM así conoces
## los atributos, etiquetas y texo....
## html.parser es la herramienta analizadora, y viene con Python. Hay otros como lxml o html5lib que son tolerantes a fallos
## resumiendo sacamos el DOM de la página.

soup = BeautifulSoup(response.text, "html.parser")


Ahora analizamos la estructura en este caso hay un bloque articel class="produc_pod" y dentro hay

* el título ``` ( <h3> <a title="..."> )  ```
* el precio ``` (<p  class="price_color" > £51.77) ```

Los buscamos y los almacenamos en un fichero .csv

In [4]:
# PASO 3. Extraer datos.

import csv

libros = soup.find_all('article', class_='product_pod')

with open("libros.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Título", "Precio"])

    for libro in libros:
        titulo = libro.h3.a['title']
        precio = libro.find('p', class_='price_color').text
        writer.writerow([titulo, precio])

print(" Datos guardados en 'libros.csv'")

# abrimos un fichero csv en modo excritura y vamos escribiendo con cada fila el título y el precio

# Dentro del for estamos guardando dos variables título y precio 

 Datos guardados en 'libros.csv'


Que pasa si hay varias páginas, en este caso es de este formato, 

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

#el formato de esta página es http://books.toscrape.com/catalogue/page-1.html
# http://books.toscrape.com/catalogue/page-2.html
#...

url_base = "http://books.toscrape.com/catalogue/page-{}.html"  ##controlamos más de una página
pagina = 1  #controla el número de página

with open("libros2.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Título", "Precio", "Disponibilidad"])

    while True:
        # el format es tomar la variable que tu especifues en ese string llaves
        url = url_base.format(pagina)
        response = requests.get(url)
        if response.status_code != 200:
            break  # No hay más páginas
            
        #en cada página necesitamos parsear 
        soup = BeautifulSoup(response.text, "html.parser")
        libros = soup.find_all('article', class_='product_pod')

        if not libros:
            break  # Si no hay libros, terminamos

        for libro in libros:
            titulo = libro.h3.a['title']
            precio = libro.find('p', class_='price_color').text
            disponibilidad = libro.find('p', class_='instock availability').text.strip()  #strip elimina espacios
            writer.writerow([titulo, precio, disponibilidad])

        pagina += 1  #variable se suma a la siguiente

print("Datos guardados en 'libros2.csv'")


Datos guardados en 'libros2.csv'


Otra forma más robusta por si falla una página. Usa **_next_**.

In [6]:
import requests
from bs4 import BeautifulSoup
import csv

url = "http://books.toscrape.com/catalogue/page-1.html"  # Página inicial

with open("libros3.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Título", "Precio", "Disponibilidad"])

    while url:  # Mientras haya una URL válida
        response = requests.get(url)
        if response.status_code != 200:
            print(f"No se pudo acceder a {url}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        libros = soup.find_all('article', class_='product_pod')

        for libro in libros:
            titulo = libro.h3.a['title']
            precio = libro.find('p', class_='price_color').text
            disponibilidad = libro.find('p', class_='instock availability').text.strip()
            writer.writerow([titulo, precio, disponibilidad])

        # Buscar enlace "next"
        next_page = soup.find('li', class_='next')
        if next_page:
            next_href = next_page.a['href']
            # Construir la URL completa de la siguiente página
            url = "/".join(url.split("/")[:-1]) + "/" + next_href
        else:
            url = None  # No hay más páginas

print("Datos guardados en 'libros3.csv'")


Datos guardados en 'libros3.csv'


Una herramienta de BeuatifulSoup es **_soup.prettify_** que nos permite ver mejor el DOM y es muy útil 
para analizar lo que queremos extreaer

In [7]:
import requests
from bs4 import BeautifulSoup

url = "http://books.toscrape.com/catalogue/page-1.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Mostrar los primeros 1000 caracteres del DOM "bonito"
print(soup.prettify()[1000:2000])

e="text/css"/>
  <link href="../static/oscar/js/bootstrap-datetimepicker/bootstrap-datetimepicker.css" rel="stylesheet"/>
  <link href="../static/oscar/css/datetimepicker.css" rel="stylesheet" type="text/css"/>
 </head>
 <body class="default" id="default">
  <header class="header container-fluid">
   <div class="page_inner">
    <div class="row">
     <div class="col-sm-8 h1">
      <a href="../index.html">
       Books to Scrape
      </a>
      <small>
       We love being scraped!
      </small>
     </div>
    </div>
   </div>
  </header>
  <div class="container-fluid page">
   <div class="page_inner">
    <ul class="breadcrumb">
     <li>
      <a href="../index.html">
       Home
      </a>
     </li>
     <li class="active">
      All products
     </li>
    </ul>
    <div class="row">
     <aside class="sidebar col-sm-4 col-md-3">
      <div id="promotions_left">
      </div>
      <div class="side_categories">
       <ul class="nav nav-list">
        <li>
         <a href="cat

O si quieres mostrar solo los libros. 

In [8]:
libros = soup.find_all('article', class_='product_pod')

for libro in libros[:2]:  # Mostrar solo los primeros 3 libros para no saturar
    print(libro.prettify())


<article class="product_pod">
 <div class="image_container">
  <a href="a-light-in-the-attic_1000/index.html">
   <img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/>
  </a>
 </div>
 <p class="star-rating Three">
  <i class="icon-star">
  </i>
  <i class="icon-star">
  </i>
  <i class="icon-star">
  </i>
  <i class="icon-star">
  </i>
  <i class="icon-star">
  </i>
 </p>
 <h3>
  <a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">
   A Light in the ...
  </a>
 </h3>
 <div class="product_price">
  <p class="price_color">
   Â£51.77
  </p>
  <p class="instock availability">
   <i class="icon-ok">
   </i>
   In stock
  </p>
  <form>
   <button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">
    Add to basket
   </button>
  </form>
 </div>
</article>

<article class="product_pod">
 <div class="image_container">
  <a href="tipping-the-velvet_999/index.html">
   <img alt="Tip

O en un navegador.

In [9]:
with open("pagina.html", "w", encoding="utf-8") as f:
    f.write(soup.prettify())


### **Proyecto BeautifulSoup**

Debes elegir una página para scrapear con BeatifulSoup y guardar los datos en formato .csv, .json o el formato que desees pero es importante que tengas en cuenta que:

* Sin login ni captchas: facilitan el scraping sin complicaciones.
* Estructura HTML clara y consistente: que puedan localizar elementos de forma fácil
* Páginas con paginación opcional, si es posible: para practicar whiles y extracción en varias páginas.
* Atentos a algún aviso de tipo legal 
* No usar e-commerce como Amazon, Ebay, usan bloqueos, mucho JavaScript y suelen prohibir scraping.
* Las redes sociales suelen dar problemas por temas de privacidad
* Lo mejor son páginas de pequeños comercios


Vuelca los datos en .json y aplícalo cuatro funciones de Pandas que no hayamos visto en clase. 

##  2. SCRAPY

**_Scrapy_** es un framework de Python para el web scraping y el crawling. Permite extraer datos estructurados de sitios web, procesarlos y guardarlos fácilmente en distintos formatos (JSON, CSV, bases de datos...).

A diferencia de *_BeautifulSoup_*, que es una librería para analizar HTML, Scrapy es un framework más completo que:

* Gestiona las peticiones HTTP.

* Controla la concurrencia (permite hacer muchas peticiones a la vez).

* Sigue enlaces automáticamente.

* Organiza el código en *_Spiders_*, *_Items_*, *_Pipelines_* y *_Middlewares_*.

* Incluye herramientas de depuración, caché y exportación de datos.

Su arquitectura es la siguiente: *_Spider → Engine → Scheduler → Downloader → Middleware → Item Pipeline_*   

**Spider** 

* Es el corazón del scraping.

* Define de dónde se parte (URL inicial) y cómo se extrae la información.

* Cada spider es una clase Python que hereda de scrapy.Spider.

**_Engine_**

* El motor **controla el flujo interno** entre los distintos componentes:

* Envía las peticiones al Scheduler.

* Recibe las respuestas del Downloader.

* Entrega los datos al Spider.

* Manda los resultados al Pipeline.

* No se programa directamente, pero es esencial para entender cómo circula la información.

**_Scheduler (Planificador)_**

* Mantiene una **cola de peticiones** pendientes.

* Decide qué URL visitar a continuación.

* Evita visitar la misma página más de una vez (usa un fingerprint).

**_Downloader (Descargador)_**

* Se encarga de hacer las **peticiones HTTP** reales.

* Usa middlewares para modificar cabeceras, retrasos, cookies, etc.

* También puede simular user agents o manejar proxies.

**_Middlewares_**

* Son capas intermedias que permiten **interceptar y modificar** peticiones o respuestas. Ejemplo: cambiar el User-Agent, usar un proxy, manejar cookies, etc.

**_Item Pipeline_**

* Se encarga del **postprocesamiento** de los datos. Puede:

* Limpiar o validar campos.

* Guardar los datos en ficheros JSON/CSV.

* Insertar en bases de datos (MongoDB, MySQL...).

**Componentes clave de un proyecto Scrapy**

* *_spiders/_*	:       Carpeta con los spiders que definen qué y cómo se extrae.
* *_items.py_*	:       Define la estructura de los datos extraídos.
* *_pipelines.py_*	:   Procesa y guarda los datos.
* *_middlewares.py_* :  Controla cómo se hacen las peticiones.
* *_settings.py_*	:   Configura el proyecto (user-agent, delays, pipelines, etc.).

Los comandos básicos son

Instalación, por si no está instlado

**_pip install scrapy_**        

Creación del proyecto, suele ponerse como nombre del proyecto algo relacionado con la web datoscoches, datosarticulos....

**_scrapy startproject nombreproyecto_**   

Ejecución y recogida de datos

**_scrapy crawl nombre_spider -o  resultados.json_**    

Resumiendo. Ciclo de Vida. Todo es asíncrono, así ningún paso bloque al otro, gracias a la herramienta Twisted un framework Python que permite gestionar particiones.

1. El Spider genera las primeras peticiones (start_requests o start_urls).

2. El Engine envía las peticiones al Scheduler, que las almacena.

3. El Engine toma una petición del Scheduler y la pasa al Downloader.

4. El Downloader ejecuta la solicitud HTTP (mediante Twisted).

5. Cuando llega la respuesta, pasa por los Downloader Middlewares.

6. El Engine entrega la respuesta al método parse() del Spider.

7. El Spider genera Items o nuevas Requests.

8. Los Items se envían al Item Pipeline, las Requests vuelven al Scheduler.

9. El ciclo continúa hasta que no quedan peticiones activas.


Esta es la **estructura básica** de un proyecto cuando lo creamos:

Supuesto práctico 

In [11]:
#Desde un terminal o un bloque de Jupyterlab con ! delante permite ejecutar comandos
!scrapy startproject citas_scraper


New Scrapy project 'citas_scraper', using template directory '/opt/conda/lib/python3.11/site-packages/scrapy/templates/project', created in:
    /home/jovyan/work/UD1/citas_scraper

You can start your first spider with:
    cd citas_scraper
    scrapy genspider example example.com


In [12]:
#entramos en citas !  y % son comando mágicos

%cd citas_scraper

/home/jovyan/work/UD1/citas_scraper


In [13]:
# creamos un spider, fíjate le pasamos la dirección

!scrapy genspider citas quotes.toscrape.com


Created spider 'citas' using template 'basic' in module:
  citas_scraper.spiders.citas


Abrimos citas desde el arbol de directorios izquierda.
Se abre en un nueva ventana y añadimos esto. 

In [ ]:
import scrapy


class CitasSpider(scrapy.Spider):
    name = "citas"  #fíjate es como llamaremos al spider
    allowed_domains = ["quotes.toscrape.com"]
    start_urls = ["https://quotes.toscrape.com"]
    # es el equivalente del this, pero le pasa todo realmetne la clase 
    def parse(self, response): #ek response es la sopa ""
            # Recorre todas las citas del bloque <div class="quote">
        for cita in response.css('div.quote'):
            yield {
                'texto': cita.css('span.text::text').get(),
                'autor': cita.css('small.author::text').get(),
                'tags': cita.css('div.tags a.tag::text').getall()
            }

        # Busca el enlace a la siguiente página
        next_page = response.css('li.next a::attr(href)').get()

        # Si existe, lo sigue
        if next_page:
            yield response.follow(next_page, callback=self.parse)


Ya solo nos queda ejecutar. Verás que var recorriendo las diferentes páginas cargando las citas. 

In [ ]:
#> log.txt 2>&1  para que no muestre pantalla solo guarde, si lo quitas lo muestra en pantalla

!scrapy crawl citas -o resultados.json  > log.txt 2>&1

Ya tenemos en un fichero json, pero ¿y si queremos mostrar en pandas? Recuerda

In [17]:
import pandas as pd

%cd /home/jovyan/work/UD1/citas_scraper
 
# Leer el fichero JSON generado por Scrapy
df = pd.read_json("resultados.json")

# Mostrar las primeras filas
df.head()

len(df)  #cuantas citas

df['autor'].unique()  #listar autores

df['autor'].value_counts()  #citas por autor

/home/jovyan/work/UD1/citas_scraper


autor
Albert Einstein           10
J.K. Rowling               9
Marilyn Monroe             7
Dr. Seuss                  6
Mark Twain                 6
C.S. Lewis                 5
Jane Austen                5
Bob Marley                 3
Eleanor Roosevelt          2
Charles Bukowski           2
Suzanne Collins            2
George R.R. Martin         2
Ralph Waldo Emerson        2
Mother Teresa              2
Ernest Hemingway           2
J.D. Salinger              1
George Bernard Shaw        1
J.R.R. Tolkien             1
Alfred Tennyson            1
Terry Pratchett            1
John Lennon                1
George Carlin              1
W.C. Fields                1
Ayn Rand                   1
Jimi Hendrix               1
J.M. Barrie                1
E.E. Cummings              1
Khaled Hosseini            1
Harper Lee                 1
Helen Keller               1
Haruki Murakami            1
Stephenie Meyer            1
Garrison Keillor           1
Thomas A. Edison           1
Douglas 

### **Proyecto Scrapy**

Existen algunas webs que permiten trabajar con scrapy. 
Es muy importante webs que tengan un **HTML muy limpio**, y en lo posible básico, y selectores CSS muy claros. 
Suelen ser supermercados, inmobiliarias, ventas de coches de ocasión pero no de grandes corpopraciones. 
Sobre todo **evitar javascript**

Desarrollar un spider Scrapy que recorra un catálogo de productos de un sitio web público y extraiga los siguientes datos, como mínimo:

Nombre o título del producto.

Precio.

Estado o disponibilidad. (si es posible)

Categoría o etiqueta (si existe).

Cualquier otro campo que consideres oportuno. 

El spider debe ser capaz de recorrer varias páginas mediante la paginación del sitio.

Los datos extraídos se almacenarán en un fichero .json o .csv para su posterior análisis en pandas.
En este punto investiga 4 o 5 funciones de Pandas que no hayamos visto.